In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression
from fredapi import Fred
import sqlite3
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import gridspec
import matplotlib.dates as mdates
from pandas.tseries.offsets import Day
from decimal import Decimal
import requests

In [2]:
# Set up your FRED API key
fred_api_key = 'bf9400b3f6a177d421bda60a77384789'  # Replace with your FRED API key
fred = Fred(api_key=fred_api_key)

In [7]:
def get_previous_trading_day():
    today = datetime.now().date()
    previous_day = today - timedelta(days=1)
    
    while previous_day.weekday() >= 5:  # 5 = Saturday, 6 = Sunday
        previous_day -= timedelta(days=1)
    
    return previous_day

# Define ticker and date range
ticker = "^GSPC"
edited_ticker = ticker.replace("^", "")
start_date = "1950-01-01"
end_date = (get_previous_trading_day() + timedelta(days=1)).strftime('%Y-%m-%d')
# end_date = "2024-11-08"

In [11]:
def fetch_market_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    print(data)
    # Create 'Syn_Open' column
    data['Syn_Open'] = data['Open']
    mask = (data['Open'] == 0) | (data['Open'].isna())
    data.loc[mask, 'Syn_Open'] = data.loc[mask, ['High', 'Low', 'Close']].mean(axis=1)
    
    # Use 'Syn_Open' where 'Open' is NaN or 0
    data['Adjusted_Open'] = data['Syn_Open']
    
    # Calculate daily returns and set the first day's return to 0
    data['Index_Returns'] = data['Adj Close'].pct_change().fillna(0)
    
    return data

# Fetch market data
market_data = fetch_market_data(ticker, start_date, end_date)

[*********************100%***********************]  1 of 1 completed
                   Open         High          Low        Close    Adj Close  \
Date                                                                          
1950-01-03    16.660000    16.660000    16.660000    16.660000    16.660000   
1950-01-04    16.850000    16.850000    16.850000    16.850000    16.850000   
1950-01-05    16.930000    16.930000    16.930000    16.930000    16.930000   
1950-01-06    16.980000    16.980000    16.980000    16.980000    16.980000   
1950-01-09    17.080000    17.080000    17.080000    17.080000    17.080000   
...                 ...          ...          ...          ...          ...   
2024-11-11  6008.859863  6017.310059  5986.689941  6001.350098  6001.350098   
2024-11-12  6003.600098  6009.919922  5960.080078  5983.990234  5983.990234   
2024-11-13  5985.750000  6008.189941  5965.910156  5985.379883  5985.379883   
2024-11-14  5989.680176  5993.879883  5942.279785  5949.169922

In [10]:
market_data

,Open,High,Low,Close,Adj Close,Volume,Syn_Open,Adjusted_Open,Index_Returns
Date,,,,,,,,,
1950-01-03,16.660000,16.660000,16.660000,16.660000,16.660000,1260000,16.660000,16.660000,0.000000
1950-01-04,16.850000,16.850000,16.850000,16.850000,16.850000,1890000,16.850000,16.850000,0.011405
1950-01-05,16.930000,16.930000,16.930000,16.930000,16.930000,2550000,16.930000,16.930000,0.004748
1950-01-06,16.980000,16.980000,16.980000,16.980000,16.980000,2010000,16.980000,16.980000,0.002953
1950-01-09,17.080000,17.080000,17.080000,17.080000,17.080000,2520000,17.080000,17.080000,0.005889
...,...,...,...,...,...,...,...,...,...
2024-11-11,6008.859863,6017.310059,5986.689941,6001.350098,6001.350098,4333000000,6008.859863,6008.859863,0.000969
2024-11-12,6003.600098,6009.919922,5960.080078,5983.990234,5983.990234,4243400000,6003.600098,6003.600098,-0.002893
2024-11-13,5985.750000,6008.189941,5965.910156,5985.379883,5985.379883,4220180000,5985.750000,5985.750000,0.000232
